# Fundamentals of Data Analysis 2021
---
### Sam Tracey
### December 2021
### Analysis of CAO points 2019 / 2020 / 2021
---

In [1]:
# Import Necessary Python Libraries.

# Regular Expressions
import re
# Convenient HTTP Requests
import requests as rq
import csv
# Working with dataframes and data
import pandas as pd
# Efficient working with datetimes
import datetime as dt

import numpy as np
# Extracting.pdf tabular data from websites
import tabula


<br>

# Import CAO 2021 Points

Reference: http://www.cao.ie/index.php?page=points&p=2021

***

In [2]:
# Retrieve CAO points URL.
resp = rq.get('http://www2.cao.ie/points/l8.php')

<br>

## Save Original CAO 2021 Data Set.

***


In [3]:
# Get The Current Date and Time
now = dt.datetime.now()

# Format as a string
nowstr =  now.strftime('%Y%m%d_%H%M%S')

In [4]:
# Create a File Path for the Original Data
path = 'data/cao2021_' + nowstr + '.html'

In [5]:
# Server is using the incorrect encoding, we need to fix it.
original_encoding = resp.encoding
# Change to CP1252
resp.encoding = 'cp1252'

In [6]:
# Save the Original html file
with open(path, 'w') as f:
    f.write(resp.text)

<br>

## Use Regular Expressions to Select Correrct Lines

***

In [7]:
#Compile Regular Expression for Matching Lines.
# Original Regular expression after week 4 videos
#re_course = re.compile('([A-Z]{2}[0-9]{3})  (.*?) (\#?|([0-9]{4}|[0-9]{3})|\*?)  (.*?)')
# Modified regular expression attempting to properly separate course details
# re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*?)([0-9]{3,5})(\*?) *')
# Third version of Regular Expression as I was missing course with no points.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')


<br>

## Loop Through the Lines of the Response and Write to .csv file

***
 

In [8]:
# This was the code I originally worked on to use REGEX to extract the data.
# I really thought it was working only to realise that it ignored any courses with 
# no points allocated in either EOS or MID.


# Define path in which to save .csv file.
# path = 'data/cao2021_csv_' + nowstr + '.csv'
# Open the csv file for writing in to.
# with open(path,'w', encoding='utf-8') as file:
    # Loop through the lines of responses.
    # for line in resp.iter_lines():
        # Match only the lines we want - those representing courses
        # if re_course.fullmatch(line.decode('cp1252')):
            # Add comma delimiters after each grouping and decode line (using incorrect decoding!)
            # csv_ver = re_course.sub(r'\1, \2, \3, \4', line.decode('cp1252'))
            # csv_ver = ' '.join(csv_ver.split())
            # csv_ver = re.sub('[#*]', '', csv_ver)
            # file.write(csv_ver + '\n')
            

In [9]:
# Noticed that my original code above was missing courses with no points

# Define path in which to save .csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv'
# Open the csv file for writing in to.
with open(path2021,'w') as file:
    # Loop through the lines of responses.
    for line in resp.iter_lines():
        # Decode the line using cp1252
        dline = line.decode('cp1252')
        # Match only the lines we want - those representing courses
        if re_course.fullmatch(dline):
            # Define Course Code
            course_code = dline[:5]
            # Define Course Title
            course_title = dline[7:58]
            # Define first points
            points1 = dline[60:66]
            # Define second points
            points2 = dline[69:75]
            # Combine all string elements into full line with comma separation
            line_join = [course_code, course_title, points1, points2]
            # Use regex.sub to replace all # and * characters with ''
            line_join = [re.sub('[#*]', '', elem) for elem in line_join]
            # Write a comma separated line to file with new line after each write.
            file.write(','.join(line_join) + '\n')

In [10]:
# Read 2021 csv to dataframe and add headers for each column
df2021 = pd.read_csv(path2021, names=['Code', 'Title', 'Points1', 'Points2'])
# Add a year column for later analysis
df2021['Year'] = 2021

In [11]:
# Have a peak at top and bottom rows of dataframe
df2021

,Code,Title,Points1,Points2,Year
0,AL801,Software Design for Virtual Reality and Gaming...,300,NaN,2021
1,AL802,Software Design in Artificial Intelligence for...,313,NaN,2021
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN,2021
3,AL805,Computer Engineering for Network Infrastructur...,321,NaN,2021
4,AL810,Quantity Surveying ...,328,NaN,2021
...,...,...,...,...,...
944,WD211,Creative Computing ...,270,NaN,2021
945,WD212,Recreation and Sport Management ...,262,NaN,2021
946,WD230,Mechanical and Manufacturing Engineering ...,230,230,2021
947,WD231,Early Childhood Care and Education ...,266,NaN,2021


<br>

## Validation 0f 2021 Data

***

To validate the data that the Python code extracted from the 2021 URL I manually copy and pasted the data from the website into a Notepad++ page. I manually removed all blank lines and institution headers to leave only lines corresponding to courses.

This text file has been saved [here](http://localhost:8888/doc/tree/data/2021_validation.txt)

You can see that the validation textfile contain 949 lines of courses which matches the number of lines returned in the .csv file produced by the Python code above.

<br>

## Reading 2020 CAO Points From Messy Excel File


Reference: http://www.cao.ie/index.php?page=points&p=2020&bb=points
***

In [12]:
# Define Path for writing Data
path2020 = 'data/cao2020_csv_' + nowstr + '.csv'

In [13]:
# Define url to read data from
Cao2020_Url = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [14]:
# Save an original version of the 2020 CAO Excel File directly from URL
# reference https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
resp = rq.get(Cao2020_Url)
output = open(path2020, 'wb')
with open(path, 'wb') as output:
    output.write(resp.content)

In [15]:
# Read 2020 CAO points from .xslx URL
df2020 = pd.read_excel(Cao2020_Url,
                   sheet_name='PointsCharts2020_V2',
                   skiprows=range(10),
                   usecols = "A:O",
                   index_col=None)
# Add a year column for later analysis
df2020['Year'] = 2020


In [16]:
# Select only lvl 8 courses
df2020 = df2020[df2020['LEVEL'] > 7]
# Have a peak at top and bottom rows of dataframe
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,LEVEL,HEI,Test/Interview #,avp,v,Year
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,8,American College,NaN,NaN,NaN,2020
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,8,American College,NaN,NaN,NaN,2020
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,8,National College of Art and Design,#,NaN,NaN,2020
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,8,National College of Art and Design,#,NaN,NaN,2020
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,8,National College of Art and Design,#,NaN,NaN,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Arts,Arts (options),WD200,AQA,NaN,AQA,NaN,AQA,NaN,336,8,Waterford Institute of Technology,NaN,avp,NaN,2020
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,8,Waterford Institute of Technology,NaN,NaN,NaN,2020
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,8,Waterford Institute of Technology,NaN,NaN,NaN,2020
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,8,Waterford Institute of Technology,NaN,NaN,NaN,2020


In [17]:
# Select only Columns of Interest
df2020 = df2020.loc[:, ['COURSE CODE2', 'COURSE TITLE', 'EOS', 'EOS Mid-point', 'Year']]

# Remove all non-numeric characters that appeared in the Points columns
df2020['EOS'] = df2020['EOS'].replace('[a-z.#*+A-Z]', '',regex=True)
df2020['EOS Mid-point'] = df2020['EOS Mid-point'].replace('[a-z.#*+A-Z]', '',regex=True)

# Rename dataframe columns
df2020.columns = ['Code', 'Title', 'Points1', 'Points2', 'Year']



In [18]:
# Write dataframe to .csv file
df2020.to_csv(path2020, encoding='utf-8', index=False)

In [19]:
# Have a peak at top and bottom rows of dataframe
df2020

,Code,Title,Points1,Points2,Year
0,AC120,International Business,209,280,2020
1,AC137,Liberal Arts,252,270,2020
2,AD101,"First Year Art & Design (Common Entry,portfolio)",,,2020
3,AD102,Graphic Design and Moving Image Design (portfo...,,,2020
4,AD103,Textile & Surface Design and Jewellery & Objec...,,,2020
...,...,...,...,...,...
1455,WD200,Arts (options),,336,2020
1460,WD210,Software Systems Development,279,337,2020
1461,WD211,Creative Computing,271,318,2020
1462,WD212,Recreation and Sport Management,270,349,2020


In [20]:
# Spot check to ensure that dataframe row 540 matches Excel row 542 (Difference in 2 = Python Zero indexing and Excel Header)
df2020.iloc[540]

Code                                        LC294
Title      Business Studies with Event Management
Points1                                       279
Points2                                       347
Year                                         2020
Name: 759, dtype: object

<br>

## Validation 0f 2020 Data

***

To validate the data that the Python code extracted from the 2020 URL I downloaded the Excel file for all CAO points from the website. I manually removed the top 10 rows which contained information pertaining to the courses but no actual course data. I then used the filter on the LEVEL column to only show the level 8 courses.This left 1028 rows which matches the number of rows returned using the Python code above.

This text file has been saved [here](http://localhost:8888/doc/tree/data/2020_Validation.xlsx)





<br>

## Reading 2019 CAO Points From PDF File
Reference: http://www.cao.ie/index.php?page=points&p=2020

***



In [35]:
# Define path for writing 2019 csv file
path2019 = 'data/cao2019_csv_' + nowstr + '.csv'

In [36]:
# Define path for writing 2019 original pdf file
path_pdf = 'data/cao2019_pdf_' + nowstr + '.pdf'

In [37]:
# Define url link for 2019 CAO points
cao2019_url = 'http://www2.cao.ie/points/lvl8_19.pdf'

In [38]:
# Save an original version of the 2019 CAO pdf File directly from URL
# reference https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
resp = rq.get(cao2019_url)
with open(path_pdf, 'wb') as output:
    output.write(resp.content)

In [25]:
# Read 2019 data in pdf format from url and convert to .csv file saving it in specified path
# reference: https://tabula-py.readthedocs.io/en/latest/getting_started.html
# tabula.convert_into(cao2019_url, path, output_format="csv", pages='all')

In [39]:
# read pdf from URL into a list class (pdf_list) then convert to dataframe
# reference: https://towardsdatascience.com/how-to-extract-tables-from-pdf-using-python-pandas-and-tabula-py-c65e43bd754
pdf_list = tabula.read_pdf(cao2019_url, lattice=True, pages='all',output_format='dataframe')
df2019=pdf_list[0]

In [41]:
# Drop # and * symbols from EOS and Mid columns
#['EOS'] =  df2019['EOS'].str.replace(r"[*#]",'')
#df2019['Mid'] =  df2019['Mid'].str.replace(r"[*#]",'')

df2019['EOS'] = df2019['EOS'].replace('[a-z.#*+A-Z]', '',regex=True)
df2019['Mid'] = df2019['Mid'].replace('[a-z.#*+A-Z]', '',regex=True)


In [42]:
# Dropping insitution header lines
# Reference: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
# Convert fields that are '' to Numpy NaN values in Course Code column
df2019['Course Code'].replace('', np.nan, inplace=True)
# Drop all lines where Course Code column contains a Nan
df2019.dropna(subset=['Course Code'], inplace=True)
# Add a year Column for later anaylsis
df2019.columns = ['Code', 'Title', 'Points1', 'Points2']
df2019['Year'] = 2019

In [43]:
# Have a peak at top and bottom rows of dataframe
df2019

,Code,Title,Points1,Points2,Year
1,AL801,Software Design with Virtual Reality and Gaming,304,328,2019
2,AL802,Software Design with Cloud Computing,301,306,2019
3,AL803,Software Design with Mobile Apps and Connected...,309,337,2019
4,AL805,Network Management and Cloud Infrastructure,329,442,2019
5,AL810,Quantity Surveying,307,349,2019
...,...,...,...,...,...
960,WD200,Arts (options),221,296,2019
961,WD210,Software Systems Development,271,329,2019
962,WD211,Creative Computing,275,322,2019
963,WD212,Recreation and Sport Management,274,311,2019


In [44]:
# Write dataframe to .csv file
df2019.to_csv(path2019, encoding='utf-8', index=False)

<br>

## Validation of 2019 Data

***

To validate the data that the Python code extracted from the 2019 URL I manually downloaded the .pdf data from the website and used Adobe Acrobat Pro 2017 to export the .pdf file to an Excel file . I deleted the top 10 rows in the Excel file then applied a filter to the Course Code. I used the filter to remove all the blank cells in the Course Code column. This left me with a total of 931 rows which matches the number of rows produced using the Python code above. I then manually compared a small sample of the two files to ensure they matched.

This Excel file used for validation has been saved [here](http://localhost:8888/doc/tree/data/2019_Validation.xlsx)

***

<br>

## Concatenation of all dataframes

***

In [45]:
# Concatenate all lvl 8 CAO points for years 2019, 2020 & 2021
df_all = pd.concat([df2019, df2020, df2021], ignore_index=True)
# Take a peak at concatenated dataframe
df_all

,Code,Title,Points1,Points2,Year
0,AL801,Software Design with Virtual Reality and Gaming,304,328,2019
1,AL802,Software Design with Cloud Computing,301,306,2019
2,AL803,Software Design with Mobile Apps and Connected...,309,337,2019
3,AL805,Network Management and Cloud Infrastructure,329,442,2019
4,AL810,Quantity Surveying,307,349,2019
...,...,...,...,...,...
2901,WD211,Creative Computing ...,270,NaN,2021
2902,WD212,Recreation and Sport Management ...,262,NaN,2021
2903,WD230,Mechanical and Manufacturing Engineering ...,230,230,2021
2904,WD231,Early Childhood Care and Education ...,266,NaN,2021


In [46]:
# List any duplicates in concatenated dataframe
df_all[df_all.duplicated()]

,Code,Title,Points1,Points2,Year


<br>

During the Deduplication of Data video we were shown a method to remove duplicates in the data based on the Course Code. When the pd.duplicated() function was run on the concatenated data set during the video, ~890 rows were shown to be duplicates. This was due to duplicated course codes across different years. I have chosen not to remove duplicate codes that occur in different years as I want to compare the increase (or otherwise) in specific courses across the past three years. Removing these duplicates would mean that I could not perform this analysis.

In [47]:
df_all.to_csv('test1.csv', encoding='utf-8', index=False)

<br>

## Joining Dataframes.

## References

[1:Real-Python_REGEX](https://realpython.com/python-web-scraping-practical-introduction/)

[2:StackOverFlow-Iter_lines](https://stackoverflow.com/questions/16870648/python-read-website-data-line-by-line-when-available)

[3:REGEX_Syntax](https://docs.python.org/3/library/re.html)

[4:StackOverFlow-utf-8](https://stackoverflow.com/questions/13110629/decoding-utf-8-strings-in-python)

[5:Understanding_ISO-8859-1](https://mincong.io/2019/04/07/understanding-iso-8859-1-and-utf-8/)
